In [1]:
import tensorflow as tf


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("MNISt_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNISt_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNISt_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNISt_data/t10k-images-idx3-ubyte.gz
Extracting MNISt_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
input_width=28
input_height=28
input_channels=1
input_pixels=784

n_conv1=32 #Convolution layer 1
k_conv1=5 #Filter size of conv1
stride_conv1=1 #Stride size for conv1

max_pool1_k=2 #Max Poolsize for pool1

n_conv2=64 #Convolution layer 2
k_conv2=5 #Filter size of conv2
stride_conv2=1 #Stride size for conv2

max_pool2_k=2 #Max Poolsize for pool2

n_hidden=1024
n_op=10

input_to_hidden=(input_width//(max_pool1_k*max_pool2_k))*(input_height//(max_pool1_k*max_pool2_k))*n_conv2

In [4]:
weights={
    "wc1":tf.Variable(tf.random_normal([k_conv1,k_conv1,input_channels,n_conv1])),
    "wc2":tf.Variable(tf.random_normal([k_conv2,k_conv2,n_conv1,n_conv2])),
    "wh":tf.Variable(tf.random_normal([input_to_hidden,n_hidden])),
    "wo":tf.Variable(tf.random_normal([n_hidden,n_op]))
}
biases={
    "bc1":tf.Variable(tf.random_normal([n_conv1])),
    "bc2":tf.Variable(tf.random_normal([n_conv2])),
    "bh":tf.Variable(tf.random_normal([n_hidden])),
    "bo":tf.Variable(tf.random_normal([n_op]))
}

In [5]:
def conv(x,weights,bias,strides=1):
    out=tf.nn.conv2d(x,weights,padding="SAME",strides=[1,strides,strides,1])
    out=tf.nn.bias_add(out,bias)
    out=tf.nn.relu(out)
    return out

def max_pooling(x,k=2):
    return tf.nn.max_pool(x,padding="SAME",ksize=[1,k,k,1],strides=[1,k,k,1])

In [6]:
def cnn(x,weights,biases):
    x=tf.reshape(x,shape=[-1,input_height,input_width,input_channels])
    conv1=conv(x,weights['wc1'],biases['bc1'],stride_conv1)
    pool1=max_pooling(conv1,max_pool1_k)
    
    conv2=conv(pool1,weights['wc2'],biases['bc2'],stride_conv2)
    pool2=max_pooling(conv2,max_pool2_k)
    
    hidden_input=tf.reshape(pool2,shape=[-1,input_to_hidden])
    hidden_input_term=tf.add(tf.matmul(hidden_input,weights['wh']),biases['bh'])
    hidden_output=tf.nn.relu(hidden_input_term)
    
    output=tf.add(tf.matmul(hidden_output,weights['wo']),biases['bo']) #using identity function at output layer
    return output
    

In [7]:
x=tf.placeholder("float",[None,input_pixels])
y=tf.placeholder(tf.int32,[None,n_op])
pred=cnn(x,weights,biases)
pred

<tf.Tensor 'Add_1:0' shape=(?, 10) dtype=float32>

In [8]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred ,labels=y))
cost

<tf.Tensor 'Mean:0' shape=() dtype=float32>

In [9]:
optimizer=tf.train.AdamOptimizer(learning_rate=0.001)
optimize=optimizer.minimize(cost)

In [10]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [11]:
##Using Batch Gradient
batch_size=100
for i in range(250):
    num_batches=int(mnist.train.num_examples/batch_size)
    total_cost=0
    for j in range(num_batches):
        batch_x,batch_y=mnist.train.next_batch(batch_size)
        c,_=sess.run([cost,optimize], feed_dict={x:batch_x, y:batch_y})
        total_cost+=c
    print(i,total_cost)

0 1772894.6029510498
1 293959.000875175
2 175019.84646511078
3 117682.71611082554
4 84850.28679212928
5 61864.130159810185
6 47181.00113943964
7 35537.559651218704
8 28441.511630814523
9 25296.62510641992
10 19477.413543647384
11 17585.541860307807
12 15736.429654978856
13 11489.283857365572
14 13924.141924135387
15 9331.275364041328
16 11394.694885075092
17 9235.137040747446
18 7512.649440016747
19 8078.455491418019
20 7206.063643977046
21 6078.958040764779
22 6288.820250868797
23 4983.4244946576655
24 4750.055147517805
25 6616.806877313248
26 4162.868663147994
27 4682.251107096672
28 4590.409471295774
29 3856.0012774169445
30 3281.9258915781975
31 3448.8366795778275
32 5132.490847527981
33 2657.0581536889076
34 2635.0391916036606
35 2733.4179230332375
36 3531.751641474664
37 2217.4821235463023
38 2944.0313799381256
39 2988.3436647093295
40 1504.0883903503418
41 2444.910477157682
42 2091.6504378986356
43 2484.243042534843
44 1589.8606748580933
45 1965.999671925325
46 2461.402328021824

In [12]:
n_pred=tf.argmax(pred,axis=1)
n_labels=tf.argmax(y,axis=1)
n_correct=tf.equal(n_pred,n_labels)

In [13]:
pred_eval,labels_eval,correct_pred=sess.run([n_pred,n_labels,n_correct], feed_dict={x:mnist.train.images, y:mnist.train.labels})
pred_eval,labels_eval,correct_pred

(array([9, 5, 4, ..., 1, 5, 9], dtype=int64),
 array([9, 5, 4, ..., 1, 5, 9], dtype=int64),
 array([ True,  True,  True, ...,  True,  True,  True]))

In [14]:
correct_pred.sum()

55000

In [15]:
#Accuracy
Accuracy=100*(correct_pred.sum()/mnist.train.num_examples)
print(Accuracy,"%")

100.0 %
